# Setup of Noteboook

The follwing code clones the github repository with course files.
Subsequently it imports all libraries and custom modules needed for this notebook

In [1]:
# import libraries
import numpy as np
import pandas as pd
import scipy
import copy
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import qmc
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')
pio.templates.default = "simple_white"
pcolors = px.colors.qualitative.T10
pcolors25 = px.colors.qualitative.Alphabet
SEED = np.random.default_rng(42)
#!git clone https://github.com/DataHow/analytics-course-scripts.git

# Process Data generation

Here we define simple in-silico process model that we will try to approximate and predict. This process model captures only the growth phase of the biomass.

The goal is to be able to predict the evolution of VCD and Glucose (or just at specific time t) given the initial conditions `VCD_0`, `GLC_0` and a process parameter `FEED_RATE`.

To approximate and predict the VCD and Glucose for an unseed parameters we first need to understand the behaviour of the process using some experimental results as training data. Subsequently, this model will allows us to predict the process behaviour for unseed validation data

In [2]:
def ode_fcn(t, y, feed_rate):
  # define parameters
    VCD = y[0]
    Glc = y[1]
    # growth rate
    mu = Glc/(5+Glc)
    # mass balances
    dVCD_dt = mu*VCD
    dGlc_dt = -(0.5*mu+0.05*Glc)*VCD+feed_rate
    dy = [dVCD_dt, dGlc_dt]
    return dy

def run_experiment(VCD_0, Glc_0, feed_rate, t_end, time_step=0.25, noise=0):
    fun = lambda t, y: ode_fcn(t,y,feed_rate)
    y0 = [VCD_0, Glc_0]
    t_span = np.arange(0, 0.5*round(2*t_end)+time_step, time_step)
    sol = solve_ivp(fun, [t_span[0], t_span[-1]], y0, method='LSODA', t_eval=t_span, rtol=1e-6, atol=1e-6)
    t = sol.t.tolist()
    y = sol.y.T
    VCD = y[:, 0] + y[:, 0]*(np.random.rand(len(t))-0.5)*2*noise
    Glc = y[:, 1] + y[:, 1]*(np.random.rand(len(t))-0.5)*2*noise
    return t, VCD, Glc

### Run single experiment

Here we generate data or experiment runs with hte insilico model above. The cell process parameters which we are able to design are following:

*   `VCD_0` Amount of VCD at time 0
*   `Glc_0` Amount of Glc at time 0
*   `feed_rate` Amount of continuous feed each day. Feeding starts at day 1.
*   `feed_end` Day when we stop feeding. This defines aslo the length of experiment.

Subsequently we plot the process evolution over time.


In [5]:
""" Set Process Parameters for run generation """
VCD_0 = .8
GLC_0 = 10
FEED_RATE = 10
FEED_END = 5
""" Measurement granularity """
TIME_STEP = 0.25
""" Measurement noise """
RELATIVE_NOISE = 0.0

In [6]:
# Generate single experiment run
t, VCD, GLC = run_experiment(VCD_0,GLC_0,FEED_RATE,FEED_END,time_step=TIME_STEP,noise=RELATIVE_NOISE)
# Plot single experiment
pcolors = px.colors.qualitative.T10
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=t,y=VCD, mode='markers+lines',name='VCD'))
fig.add_trace(go.Scatter(x=t,y=GLC, mode='markers+lines',name='Glucose'),secondary_y=True)
fig.update_layout(showlegend=True,title="Cell process dynamics",xaxis_title="Time",yaxis_title="Glc",width=1000)
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()

### Run experimental design

Here we use the same insilico model as above, but specify ranges for each process parameters. A design of experiments will be performed within these parameters ranges using Latin Hypercube Sampling for the specified amount of experiments.

This will serve as a training data for our process model.

In [7]:
""" Number of experiments in training """
NUM_RUNS = 4
""" Define initial conditions for process parameters in DOE """
VCD_0_BOUNDS = [0.5,3.5]
GLC_0_BOUNDS = [10,30]
FEED_RATE_BOUNDS = [10,50]

In [8]:
# Create Design of Experiments
doe_train = np.zeros((NUM_RUNS,3))
sampler = qmc.LatinHypercube(d=3,seed=42,centered=True)
doe_train_nondim = 2*sampler.random(n=NUM_RUNS)-1
# Transform DOE from [-1,1] range to the specified bounds
doe_train[:,0] = ((doe_train_nondim[:,0]+1)/2)*(VCD_0_BOUNDS[1] - VCD_0_BOUNDS[0]) + VCD_0_BOUNDS[0]
doe_train[:,1] = ((doe_train_nondim[:,1]+1)/2)*(GLC_0_BOUNDS[1] - GLC_0_BOUNDS[0]) + GLC_0_BOUNDS[0]
doe_train[:,2] = ((doe_train_nondim[:,2]+1)/2)*(FEED_RATE_BOUNDS[1] - FEED_RATE_BOUNDS[0]) + FEED_RATE_BOUNDS[0]
LEN_RUNS = int(np.ceil((FEED_END/TIME_STEP)+1))
# Generate experiments
vcd = np.zeros((LEN_RUNS,NUM_RUNS))
glc = np.zeros((LEN_RUNS,NUM_RUNS))
for i in range(NUM_RUNS):
    t, vcd[:,i],glc[:,i] = run_experiment(doe_train[i,0],doe_train[i,1],doe_train[i,2],FEED_END,time_step=TIME_STEP,noise=RELATIVE_NOISE)
# Plot experiments
fig = make_subplots(specs=[[{"secondary_y": True}]])
for i in range(NUM_RUNS):
    fig.add_trace(go.Scatter(x=t,y=vcd[:, i], mode='markers+lines',name=f'VCD | run {i}', line_color=pcolors[0],legendgroup=i))
    fig.add_trace(go.Scatter(x=t,y=glc[:, i], mode='markers+lines',name=f'Glc | run {i}',line_color=pcolors[1],legendgroup=i),secondary_y=True)
fig.update_layout(showlegend=True,title="Cell process dynamics in DOE",xaxis_title="Time",yaxis_title="Glc",width=1000)
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()

### Create a validation set


In similar fashion we create a validation set with significantly larger number of experiments to properly verify the model's performance on a completely unseen data.

In [9]:
""" Number of experiments in validation """
NUM_TEST = 100

In [10]:
# Create design of experiment with more runs for validation
doe_test = np.zeros((NUM_TEST,3))
sampler = qmc.LatinHypercube(d=3, seed=42)
doe_test_nondim = 2*sampler.random(n=NUM_TEST)-1
# Transform DOE from [-1,1] range to the specified bounds
doe_test[:,0] = ((doe_test_nondim[:,0]+1)/2)*(VCD_0_BOUNDS[1] - VCD_0_BOUNDS[0]) + VCD_0_BOUNDS[0]
doe_test[:,1] = ((doe_test_nondim[:,1]+1)/2)*(GLC_0_BOUNDS[1] - GLC_0_BOUNDS[0]) + GLC_0_BOUNDS[0]
doe_test[:,2] = ((doe_test_nondim[:,2]+1)/2)*(FEED_RATE_BOUNDS[1] - FEED_RATE_BOUNDS[0]) + FEED_RATE_BOUNDS[0]
LEN_RUNS = int(np.ceil((FEED_END/TIME_STEP)+1))
# Generate experiments
vcd_test = np.zeros((LEN_RUNS,NUM_TEST))
glc_test = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    t, vcd_test[:,i],glc_test[:,i] = run_experiment(doe_test[i,0],doe_test[i,1],doe_test[i,2],FEED_END,time_step=TIME_STEP,noise=RELATIVE_NOISE)
# Plot experiments
fig = make_subplots(specs=[[{"secondary_y": True}]])
for i in range(NUM_TEST):
    fig.add_trace(go.Scatter(x=t,y=vcd_test[:, i], mode='markers+lines',name=f'VCD | run {i}', line_color=pcolors[0]))
    fig.add_trace(go.Scatter(x=t,y=glc_test[:, i], mode='markers+lines',name=f'Glc | run {i}',line_color=pcolors[1]),secondary_y=True)
fig.update_layout(showlegend=False,title="Cell process dynamics in DOE | Validation set",xaxis_title="Time",yaxis_title="Glc",width=1000)
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()

# Non-Hybrid Models

In this section we demonstrate how a non-hybrid model for the insilico data could look like. We first create a very simple mechanistic model that approximates the data using exponential growth. Secondly, a pure data-driven model doesn't try to model the process evolution but directly predicts VCD at time t.


## Mechanistic Model


Let's suppose the following simple model describing exponential growth for `VCD`:

$$
\frac{\mathrm{dVCD}}{\mathrm{dt}}=g*\mathrm{VCD}
$$

This is a first-order linear ordinary differencial equation. We can easily solve it an the solution has familiar form of

$$
VCD(t) = A*\mathrm{exp}(gt)
$$


Functions needs to learn the combination of the intrinsic process behavior, like GLC specific consumption with the total amount of cells. So for GLC it estimates the **global** glucose consumption

To obtain the full profiles in prediction, we use reconstruct it following way for each timestep:

$$
VCD_{t + \Delta t} = VCD_{t} + \frac{\mathrm{dVCD}}{\mathrm{dt}}\Delta t
$$


Let's use the inverse method to get the model. First, let's estimate the derivatives for each step individual using the approximation of subsequtive steps:
$$
\frac{\mathrm{dVCD}}{\mathrm{dt}} ≈ ΔVCD_t \approx VCD_{t + \Delta t} - VCD_{t}
$$

## Data driven model

### Train and predict model

In [11]:
# Define Model
kernel = 1.0 * RBF(length_scale=1e-1, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-10, 1e1))
vcd_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)
glc_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)

# Train Model
t_idx = [t.index(i) for i in t]
vcd_dd_mdl = []
glc_dd_mdl = []
for i,j in enumerate(t_idx):
    vcd_dd_mdl.append(copy.copy(vcd_mdl).fit(doe_train,vcd[j,:]))
    glc_dd_mdl.append(copy.copy(glc_mdl).fit(doe_train,glc[j,:]))

# Prediction in train set
LEN_STEPS = len(t)
vcd_train_pred = np.zeros((LEN_STEPS, NUM_RUNS))
glc_train_pred = np.zeros((LEN_STEPS, NUM_RUNS))
for i in range(NUM_RUNS):
    for j in range(LEN_STEPS):
        vcd_train_pred[j,i] = vcd_dd_mdl[j].predict(doe_train[i,:].reshape(1,-1))
        glc_train_pred[j,i] = glc_dd_mdl[j].predict(doe_train[i,:].reshape(1,-1))

# Prediction in test set
vcd_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
glc_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    for j in range(LEN_STEPS):
        vcd_test_pred[j,i] = vcd_dd_mdl[j].predict(doe_test[i,:].reshape(1,-1))
        glc_test_pred[j,i] = glc_dd_mdl[j].predict(doe_test[i,:].reshape(1,-1))
vcd_test_rmse = mean_squared_error(vcd_test, vcd_test_pred,squared=False) / np.std(np.array(vcd))
glc_test_rmse = mean_squared_error(glc_test, glc_test_pred,squared=False) / np.std(np.array(vcd))
vcd_test_r2 = r2_score(vcd_test,vcd_test_pred)
glc_test_r2 = r2_score(glc_test,glc_test_pred)

### Evaluate model

In [12]:
""" Select experiment on which to predict """
PLOT_TRAIN=0
PLOT_TEST=50

# Plot selected experiment run
fig = make_subplots(rows=1,cols=2,specs=[[{"secondary_y": True},{"secondary_y": True}]],subplot_titles=(f"Train set - run {PLOT_TRAIN} - prediction",f"Test set - run {PLOT_TEST} - prediction"))
# Train Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd[:,PLOT_TRAIN], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_train_pred[:,PLOT_TRAIN],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc[:,PLOT_TRAIN], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=1)
fig.add_trace(go.Scatter(x=t, y=glc_train_pred[:,PLOT_TRAIN],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=1)
# Test Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd_test[:,PLOT_TEST], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_test_pred[:,PLOT_TEST],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc_test[:,PLOT_TEST], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=2)
fig.add_trace(go.Scatter(x=t, y=glc_test_pred[:,PLOT_TEST],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=2)
fig.update_layout(showlegend=True,title="Predicted Profile: Glc and VCD over time for run id: "+str(PLOT_TRAIN)+" in train set (left) & "+str(PLOT_TEST)+" in test set (right)", height=500)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()
# Plot observed vs predicted
fig = make_subplots(rows=1, cols=2, subplot_titles=[
    f'Test set - all runs - VCD <br> ( rmse = {vcd_test_rmse:.3f}, r2 = {vcd_test_r2:.3f} )',
    f'Test set - all runs - Glc <br> ( rmse = {glc_test_rmse:.3f}, r2 = {glc_test_r2:.3f} )'])
fig.add_shape(type="line",x0=0,y0=0,x1=150,y1=150,row=1,col=2)
for i in range(NUM_TEST):
    fig.add_trace(go.Scatter(x=vcd_test[:,i],y=vcd_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=1)
    fig.add_trace(go.Scatter(x=glc_test[:,i],y=glc_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=2)
fig.add_shape(type="line",x0=0,y0=0,x1=150,y1=150, line_color='black',line_dash='dash',line_width=2,row=1,col=1)

fig.add_shape(type="line",x0=vcd_test.min(),y0=vcd_test.min(),x1=vcd_test.max(),y1=vcd_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=1)
fig.add_shape(type="line",x0=glc_test.min(),y0=glc_test.min(),x1=glc_test.max(),y1=glc_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=2)
fig.update_layout(title_text = "Observed vs Predicted: ",showlegend=True, height=500)
fig.update_xaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_yaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_xaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.update_yaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.show()

# Hybrid Models

We use a simple model for cell expansion, accounting for the growth of viable cell density (VCD) and the consumption of glucose (Glc), is used to show how hybrid models are trained and can predict new experimental conditions.

Furthermore we define functions specifying our two hybrid models below which will be utilized further in the script. They consist of two functions. First function defines the ODE equations that describe the dynamics and second function solves the equations using specified parameters and initial conditions.

## First Hybrid Model  

Let's suppose the following model (VCD dependent):

$$
\begin{aligned}
&\frac{\mathrm{dVCD}}{\mathrm{dt}}=g(\mathrm{VCD}, \mathrm{Glc}) \\
&\frac{\mathrm{dGlc}}{\mathrm{dt}}=-k(\mathrm{VCD}, \mathrm{Glc})+\text { feed }
\end{aligned}
$$

Functions needs to learn the combination of the intrinsic process behavior, like GLC specific consumption with the total amount of cells. So for GLC it estimates the **global** glucose consumption

To obtain the full profiles in prediction, we use reconstruct it following way for each timestep:

$$
VCD_{t + \Delta t} = VCD_{t} + \frac{\mathrm{dVCD}}{\mathrm{dt}}\Delta t
$$


Let's use the inverse method to get the model. First, let's estimate the derivatives for each step individual using the approximation of subsequtive steps:
$$
\frac{\mathrm{dVCD}}{\mathrm{dt}} ≈ ΔVCD_t \approx VCD_{t + \Delta t} - VCD_{t}
$$

### Find and visualize derivatives

In [13]:
# Estimating derivatives (changes/deltas)
g = np.zeros((LEN_RUNS, NUM_RUNS))
k = np.zeros((LEN_RUNS, NUM_RUNS))
# initial derivatives
g[0,:] = (vcd[1,:]-vcd[0,:])/TIME_STEP
dGlcdt = (glc[1,:]-glc[0,:])/TIME_STEP
k[0,:] = -dGlcdt+doe_train[:,2].T
# timestep derivatives
for j in range(1,LEN_RUNS-1):
    g[j,:] = 0.5*(vcd[j+1,:]-vcd[j-1,:])/TIME_STEP
    dGlcdt = 0.5*(glc[j+1,:]-glc[j-1,:])/TIME_STEP
    k[j,:] = -dGlcdt+doe_train[:,2].T
# final derivatives
g[-1,:] = (vcd[-1,:]-vcd[-2,:])/TIME_STEP
dGlcdt = (glc[-1,:]-glc[-2,:])/TIME_STEP
k[-1,:] = -dGlcdt+doe_train[:,2].T
# collect all the inputs
X = np.stack((vcd.flatten(), glc.flatten()), axis=1)

In [14]:
# Visualize the derivatives for VCD
print(pd.DataFrame(g))

            0          1          2          3
0    2.964075   2.235436   1.371875   0.777869
1    3.334561   2.564005   1.594080   0.888583
2    4.164292   3.296653   2.078211   1.136050
3    5.190419   4.206394   2.663136   1.443881
4    6.452664   5.348083   3.390047   1.830063
5    7.993046   6.784539   4.301260   2.316144
6    9.850502   8.590419   5.446060   2.928571
7   12.049551  10.853846   6.883355   3.699945
8   14.577967  13.675374   8.683030   4.670276
9   17.345891  17.161764  10.925724   5.888119
10  20.120274  21.408407  13.699273   7.411291
11  22.453075  26.458345  17.088073   9.306399
12  23.712594  32.216810  21.147636  11.645678
13  23.444988  38.301371  25.849828  14.497951
14  21.971876  43.880326  30.977876  17.907451
15  20.316050  47.815635  35.969321  21.847848
16  19.241007  49.540368  39.842739  26.132245
17  18.777625  49.850918  41.645460  30.267159
18  18.646505  50.033962  41.505776  33.339559
19  18.641934  50.562730  40.749530  34.337945
20  18.654840

In [15]:
# Plot scatterplots for VCD and GLC, color by derivative
fig = make_subplots(rows=2, cols=2, subplot_titles=("VCD over time - VCD growth rate","GLC over time - GLC consumption rate","VCD vs GLC - VCD growth rate","VCD vc GLC - GLC consumption rate" ))
fig.add_trace(go.Scatter(x=np.repeat(t,NUM_RUNS),y=vcd.flatten(),mode='markers',marker=dict(color=g.flatten(),colorscale=px.colors.sequential.Viridis,showscale=True,colorbar=dict(len=1.0, x=0.45)),),row=1,col=1)
fig.add_trace(go.Scatter(x=np.repeat(t,NUM_RUNS),y=glc.flatten(),mode='markers',marker=dict(color=k.flatten(),colorscale=px.colors.sequential.Viridis,showscale=True,colorbar=dict(len=1.0, x=1.0)),),row=1,col=2)
fig.add_trace(go.Scatter(x=vcd.flatten(),y=glc.flatten(),mode='markers',marker=dict(color=g.flatten(),colorscale=px.colors.sequential.Viridis),),row=2,col=1)
fig.add_trace(go.Scatter(x=vcd.flatten(),y=glc.flatten(),mode='markers',marker=dict(color=k.flatten(),colorscale=px.colors.sequential.Viridis),),row=2,col=2)
fig.update_layout(title_text = "State plots of process colored by growth/consumption rates",showlegend=False,height=1000)
fig.update_xaxes(title_text="Time", row=1, col=1), fig.update_yaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=1, col=1)
fig.update_xaxes(title_text="Time", row=1, col=2), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=1, col=2)
fig.update_xaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=2, col=1), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=2, col=1)
fig.update_xaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=2, col=2), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=2, col=2)
fig.show()

### Train and predict model

Let's fit the same model, but training on the calculated derivatives:

In [16]:
# Define Model
kernel = 1.0 * RBF(length_scale=1e-1, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-10, 1e1))
g_1h_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)
k_1h_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)

def ode_fcn_1st_hybrid(t, y, feed, g_mld, k_mld):
    # mass balances
    dVCD_dt = g_mld.predict(y.reshape(-1, 1).T)
    dGlc_dt = -k_mld.predict(y.reshape(-1, 1).T) + feed
    return [dVCD_dt, dGlc_dt]

def run_1st_hybrid(VCD_0, Glc_0, feed, t_end, g_mld, k_mld,time_step =0.25):
    fun = lambda t, y: ode_fcn_1st_hybrid(t, y, feed, g_mld, k_mld)
    y0 = np.array([VCD_0, Glc_0])
    t_span = np.arange(0, 0.5 * round(2 * t_end) + time_step, time_step)
    sol = solve_ivp(fun, [t_span[0], t_span[-1]], y0, method='LSODA', t_eval=t_span, rtol=1e-6, atol=1e-6)
    t = sol.t.tolist()
    y = sol.y.T
    VCD = y[:, 0]
    Glc = y[:, 1]
    return t, VCD, Glc

# Train Model
g_1h_mdl.fit(X, g.flatten())
k_1h_mdl.fit(X, k.flatten())

# Predictions in train set
vcd_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
glc_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
for i in range(NUM_RUNS):
    t,vcd_train_pred[:,i], glc_train_pred[:,i] = run_1st_hybrid(doe_train[i,0], doe_train[i,1], doe_train[i,2], FEED_END, g_1h_mdl, k_1h_mdl, time_step=TIME_STEP)
# Predictions in test set
vcd_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
glc_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    t, vcd_test_pred[:,i], glc_test_pred[:,i] = run_1st_hybrid(doe_test[i,0], doe_test[i,1], doe_test[i,2], FEED_END, g_1h_mdl, k_1h_mdl, time_step=TIME_STEP)

vcd_test_rmse = mean_squared_error(vcd_test, vcd_test_pred,squared=False) / np.std(np.array(vcd))
glc_test_rmse = mean_squared_error(glc_test, glc_test_pred,squared=False) / np.std(np.array(vcd))
vcd_test_r2 = r2_score(vcd_test,vcd_test_pred)
glc_test_r2 = r2_score(glc_test,glc_test_pred)


### Evaluate model

In [17]:
""" Select experiment on which to predict """
PLOT_TRAIN=0
PLOT_TEST=50

# Plot selected experiment run
fig = make_subplots(rows=1,cols=2,specs=[[{"secondary_y": True},{"secondary_y": True}]],subplot_titles=(f"Train set - run {PLOT_TRAIN} - prediction",f"Test set - run {PLOT_TEST} - prediction"))
# Train Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd[:,PLOT_TRAIN], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_train_pred[:,PLOT_TRAIN],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc[:,PLOT_TRAIN], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=1)
fig.add_trace(go.Scatter(x=t, y=glc_train_pred[:,PLOT_TRAIN],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=1)
# Test Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd_test[:,PLOT_TEST], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_test_pred[:,PLOT_TEST],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc_test[:,PLOT_TEST], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=2)
fig.add_trace(go.Scatter(x=t, y=glc_test_pred[:,PLOT_TEST],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=2)
fig.update_layout(showlegend=True,title="Predicted Profile: Glc and VCD over time for run id: "+str(PLOT_TRAIN)+" in train set (left) & "+str(PLOT_TEST)+" in test set (right)", height=500)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()
# Plot observed vs predicted
fig = make_subplots(rows=1, cols=2, subplot_titles=[
    f'Test set - all runs - VCD <br> ( rmse = {vcd_test_rmse:.3f}, r2 = {vcd_test_r2:.3f} )',
    f'Test set - all runs - Glc <br> ( rmse = {glc_test_rmse:.3f}, r2 = {glc_test_r2:.3f} )'])
fig.add_shape(type="line",x0=0,y0=0,x1=150,y1=150,row=1,col=2)
for i in range(NUM_TEST):
    fig.add_trace(go.Scatter(x=vcd_test[:,i],y=vcd_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=1)
    fig.add_trace(go.Scatter(x=glc_test[:,i],y=glc_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=2)
fig.add_shape(type="line",x0=0,y0=0,x1=150,y1=150, line_color='black',line_dash='dash',line_width=2,row=1,col=1)

fig.add_shape(type="line",x0=vcd_test.min(),y0=vcd_test.min(),x1=vcd_test.max(),y1=vcd_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=1)
fig.add_shape(type="line",x0=glc_test.min(),y0=glc_test.min(),x1=glc_test.max(),y1=glc_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=2)
fig.update_layout(title_text = "Observed vs Predicted: ",showlegend=True, height=500)
fig.update_xaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_yaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_xaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.update_yaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.show()

We can see that the model performs generally very good besides few experiments.

## Second Hybrid Model

Let's suppose the following model (VCD independent):

$$
\begin{aligned}
&\frac{\mathrm{dVCD}}{\mathrm{dt}}=\mu(\mathrm{VCD}, \mathrm{Glc}) \cdot \mathrm{VCD} \\
&\frac{\mathrm{dGlc}}{\mathrm{dt}}=-c(\mathrm{VCD}, \mathrm{Glc}) \cdot \mathrm{VCD}+\text { feed }
\end{aligned}
$$

Functions learns intrinsic process behavior of each cell, such as how much GLC it consumes. Then, this function is multiplied by the overall amount of cells in the process.


To obtain the full profiles in prediction, we use reconstruct it following way for each timestep:

$$
VCD_{t + \Delta t} = VCD_{t} + \frac{\mathrm{dVCD}}{\mathrm{dt}}\Delta t
$$


As before, let's use the inverse method to get the model. First, let's estimate the derivatives for each step individual using the approximation of subsequtive steps:
$$
\frac{\mathrm{dVCD}}{\mathrm{dt}} ≈ ΔVCD_t \approx VCD_{t + \Delta t} - VCD_{t}
$$


### Find and model derivatives

In [18]:
# Estimating derivatives (changes/deltas)
mu = np.zeros((LEN_RUNS, NUM_RUNS))
c = np.zeros((LEN_RUNS, NUM_RUNS))
# initial derivatives
dVcddt = (vcd[1,:]-vcd[0,:])/TIME_STEP
mu[0,:] = dVcddt/vcd[0,:]
dGlcdt = (glc[1,:]-glc[0,:])/TIME_STEP
c[0,:] = (-dGlcdt+doe_train[:,2].T)/vcd[1,:]
# timestep derivatives
for j in range(1,LEN_RUNS-1):
    dVcddt = 0.5*(vcd[j+1,:]-vcd[j-1,:])/TIME_STEP
    mu[j,:] = dVcddt/vcd[j,:]
    dGlcdt = 0.5*(glc[j+1,:]-glc[j-1,:])/TIME_STEP
    c[j,:] = (-dGlcdt+doe_train[:,2].T)/vcd[j,:]
# final derivatives
dVcddt = (vcd[-1,:]-vcd[-2,:])/TIME_STEP
mu[-1,:] = dVcddt/vcd[-1,:]
dGlcdt = (glc[-1,:]-glc[-2,:])/TIME_STEP
c[-1,:] = (-dGlcdt+doe_train[:,2].T)/vcd[-1,:]

In [19]:
# Visualize the derivatives for VCD
print(pd.DataFrame(mu))

           0         1         2         3
0   0.948504  0.941236  0.844231  0.888993
1   0.862531  0.873936  0.810013  0.830865
2   0.868958  0.901463  0.858042  0.861106
3   0.872608  0.917989  0.885623  0.881764
4   0.873458  0.928454  0.903144  0.896548
5   0.871224  0.934995  0.914753  0.907394
6   0.865293  0.938590  0.922399  0.915378
7   0.854593  0.939610  0.927035  0.921116
8   0.837391  0.937993  0.929072  0.924940
9   0.810983  0.933256  0.928534  0.926979
10  0.771432  0.924359  0.925078  0.927184
11  0.713955  0.909430  0.917910  0.925311
12  0.635585  0.885334  0.905571  0.920859
13  0.541390  0.847343  0.885568  0.912944
14  0.448124  0.790065  0.853909  0.900075
15  0.374206  0.712158  0.805060  0.879750
16  0.325079  0.623558  0.734261  0.847914
17  0.293806  0.542376  0.644663  0.798600
18  0.271904  0.479375  0.552783  0.725514
19  0.254550  0.432422  0.477422  0.629245
20  0.239476  0.392603  0.423491  0.541930


Let's plot the data:

In [20]:
# Plot scatterplots for VCD and GLC, color by derivative
fig = make_subplots(rows=2, cols=2, subplot_titles=("VCD over time - VCD spec growth rate","GLC over time - GLC spec consumption rate","VCD vs GLC - VCD spec growth rate","VCD vc GLC - GLC spec consumption rate" ))
fig.add_trace(go.Scatter(x=np.repeat(t,NUM_RUNS),y=vcd.flatten(),mode='markers',marker=dict(color=mu.flatten(),colorscale=px.colors.sequential.Viridis,showscale=True,colorbar=dict(len=1.0, x=0.45)),),row=1,col=1)
fig.add_trace(go.Scatter(x=np.repeat(t,NUM_RUNS),y=glc.flatten(),mode='markers',marker=dict(color=c.flatten(),colorscale=px.colors.sequential.Viridis,showscale=True,colorbar=dict(len=1.0, x=1.0)),),row=1,col=2)
fig.add_trace(go.Scatter(x=vcd.flatten(),y=glc.flatten(),mode='markers',marker=dict(color=mu.flatten(),colorscale=px.colors.sequential.Viridis),),row=2,col=1)
fig.add_trace(go.Scatter(x=vcd.flatten(),y=glc.flatten(),mode='markers',marker=dict(color=c.flatten(),colorscale=px.colors.sequential.Viridis),),row=2,col=2)
fig.update_layout(title_text = "State plots of process colored by growth/consumption rates",showlegend=False,height=1000)
fig.update_xaxes(title_text="Time", row=1, col=1), fig.update_yaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=1, col=1)
fig.update_xaxes(title_text="Time", row=1, col=2), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=1, col=2)
fig.update_xaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=2, col=1), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=2, col=1)
fig.update_xaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=2, col=2), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=2, col=2)
fig.show()

Let's fit a model:

In [21]:
# Define Model
kernel = 1.0 * RBF(length_scale=1e-1, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-10, 1e1))
mu_2h_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)
c_2h_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)

def ode_fcn_2nd_hybrid(t, y, feed, mu_mld, c_mld):
    # mass balances
    dVCD_dt = mu_mld.predict(y.reshape(-1, 1).T)*y[0]
    dGlc_dt = -c_mld.predict(y.reshape(-1, 1).T)*y[0]+feed
    return [dVCD_dt, dGlc_dt]


def run_2nd_hybrid(VCD_0,Glc_0,feed,t_end,mu_mld,c_mld,time_step=0.25):
    fun = lambda t, y: ode_fcn_2nd_hybrid(t, y, feed, mu_mld, c_mld)
    y0 = np.array([VCD_0, Glc_0])
    t_span = np.arange(0, 0.5 * round(2 * t_end) + time_step, time_step)
    sol = solve_ivp(fun, [t_span[0], t_span[-1]], y0, method='LSODA', t_eval=t_span, rtol=1e-6, atol=1e-6)
    t = sol.t.tolist()
    y = sol.y.T
    VCD = y[:, 0]
    Glc = y[:, 1]
    return t, VCD, Glc

# Train Model
mu_2h_mdl.fit(X, mu.flatten())
c_2h_mdl.fit(X, c.flatten())

# Predictions in train set
vcd_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
glc_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
for i in range(NUM_RUNS):
    t,vcd_train_pred[:,i], glc_train_pred[:,i] = run_2nd_hybrid(doe_train[i,0], doe_train[i,1], doe_train[i,2], FEED_END, mu_2h_mdl, c_2h_mdl, time_step=TIME_STEP)
# Predictions in test set
vcd_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
glc_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    t, vcd_test_pred[:,i], glc_test_pred[:,i] = run_2nd_hybrid(doe_test[i,0], doe_test[i,1], doe_test[i,2], FEED_END, mu_2h_mdl, c_2h_mdl, time_step=TIME_STEP)

vcd_test_rmse = mean_squared_error(vcd_test, vcd_test_pred,squared=False) / np.std(np.array(vcd))
glc_test_rmse = mean_squared_error(glc_test, glc_test_pred,squared=False) / np.std(np.array(vcd))
vcd_test_r2 = r2_score(vcd_test,vcd_test_pred)
glc_test_r2 = r2_score(glc_test,glc_test_pred)


### Evaluate model

In [22]:
vcd_test_r2

0.9870236420061272

In [23]:
glc_test_rmse

0.04363819619716803

In [24]:
X = np.stack((vcd.flatten(), glc.flatten()), axis=1)
#kernel2 = 1.0 * RBF(length_scale=1e-1, length_scale_bounds=(1e-3, 1e3)) + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-10, 1e1))
g_mld = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0.0, n_restarts_optimizer=3).fit(X, g.flatten())
k_mld = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0.0, n_restarts_optimizer=3).fit(X, k.flatten())

### Model Evaluation on Train and Test Set

Make prediction on the training set and testing set:


In [25]:
# Predictions in train set
vcd_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
glc_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
for i in range(NUM_RUNS):
    t,vcd_train_pred[:,i],glc_train_pred[:,i] = run_2nd_hybrid(doe_train[i,0], doe_train[i,1], doe_train[i,2], FEED_END, g_mld, k_mld, time_step=TIME_STEP)
# Predictions in test set
vcd_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
glc_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    t, vcd_test_pred[:,i], glc_test_pred[:,i] = run_2nd_hybrid(doe_test[i,0], doe_test[i,1], doe_test[i,2], FEED_END, g_mld, k_mld, time_step=TIME_STEP)

Plot the fit of our model (2nd hybrid) over time:

In [26]:
""" Select experiment on which to predict """
PLOT_TRAIN=0
PLOT_TEST=49

In [27]:
# Plot selected experiment run
fig = make_subplots(rows=1,cols=2,specs=[[{"secondary_y": True},{"secondary_y": True}]],subplot_titles=("Train Set prediction","Test Set prediction"))
fig.add_trace(go.Scatter(x=t,y=vcd_train_pred[:,PLOT_TRAIN], mode='lines',name='VCD Predicted',line=dict(color = px.colors.qualitative.G10[0], dash='dash')),row=1,col=1)
fig.add_trace(go.Scatter(x=t,y=vcd[:,PLOT_TRAIN], mode='markers+lines',name='VCD Measured',line=dict(color = px.colors.qualitative.G10[0])),row=1,col=1)
fig.add_trace(go.Scatter(x=t,y=glc_train_pred[:,PLOT_TRAIN], mode='lines',name='Glucose Predicted',line=dict(color = px.colors.qualitative.G10[1], dash='dash')),secondary_y=True,row=1,col=1)
fig.add_trace(go.Scatter(x=t,y=glc[:,PLOT_TRAIN], mode='markers+lines',name='Glucose Measured',line=dict(color = px.colors.qualitative.G10[1])),secondary_y=True,row=1,col=1)
fig.add_trace(go.Scatter(x=t,y=vcd_test_pred[:,PLOT_TEST], mode='lines',name='VCD Predicted',line=dict(color = px.colors.qualitative.G10[0], dash='dash')),row=1,col=2)
fig.add_trace(go.Scatter(x=t,y=vcd_test[:,PLOT_TEST], mode='markers+lines',name='VCD Measured',line=dict(color = px.colors.qualitative.G10[0])),row=1,col=2)
fig.add_trace(go.Scatter(x=t,y=glc_test_pred[:,PLOT_TEST], mode='lines',name='Glucose Predicted',line=dict(color = px.colors.qualitative.G10[1], dash='dash')),secondary_y=True,row=1,col=2)
fig.add_trace(go.Scatter(x=t,y=glc_test[:,PLOT_TEST], mode='markers+lines',name='Glucose Measured',line=dict(color = px.colors.qualitative.G10[1])),secondary_y=True,row=1,col=2)
fig.update_layout(showlegend=True,title="Predicted profile Glc and VCD profile over time for run id: "+str(PLOT_TRAIN)+" in train set (left) & "+str(PLOT_TEST)+" in test set (right)",xaxis_title="Time",yaxis_title="Glc")
fig.update_yaxes(title_text="VCD", secondary_y=True)
fig.show()

Plot Observed vs Predicted for (2nd hybrid):

In [28]:
# Plot observed vs predicted
fig = make_subplots(rows=2, cols=2, subplot_titles=(
    f"VCD - Train Set <br> R^2 = {round(r2_score(vcd,vcd_train_pred),3)} <br> Rel RMSE = {round(mean_squared_error(vcd, vcd_train_pred,squared=False) / np.std(np.array(vcd)),3)}" ,
    f"VCD - Test Set <br> R^2 = {round(r2_score(vcd_test,vcd_test_pred),3)} <br> Rel RMSE = {round(mean_squared_error(vcd_test, vcd_test_pred,squared=False) / np.std(np.array(vcd)),3)}" ,
    f"GLC - Train Set <br> R^2 = {round(r2_score(glc,glc_train_pred),3)} <br> Rel RMSE = {round(mean_squared_error(glc, glc_train_pred,squared=False) / np.std(np.array(glc)),3)}" ,
    f"GLC - Test Set <br> R^2 = {round(r2_score(glc_test,glc_test_pred),3)} <br> Rel RMSE = {round(mean_squared_error(glc_test, glc_test_pred,squared=False) / np.std(np.array(glc)),3)}"))
for i in range(NUM_RUNS):
    fig.add_trace(go.Scatter(x=vcd[:,i],y=vcd_train_pred[:,i],mode="markers",name=f"run id {i}"),row=1,col=1)
    fig.add_trace(go.Scatter(x=glc[:,i],y=glc_train_pred[:,i],mode="markers",name=f"run id {i}"),row=2,col=1)
fig.add_shape(type="line",x0=vcd.min(),y0=vcd.min(),x1=vcd.max(),y1=vcd.max(), layer='above', line=dict(dash='dash'),row=1,col=1)
fig.add_shape(type="line",x0=glc.min(),y0=glc.min(),x1=glc.max(),y1=glc.max(), layer='above', line=dict(dash='dash'),row=2,col=1)
for i in range(NUM_TEST):
    fig.add_trace(go.Scatter(x=vcd_test[:,i],y=vcd_test_pred[:,i],mode="markers",name=f"run id {i}"),row=1,col=2)
    fig.add_trace(go.Scatter(x=glc_test[:,i],y=glc_test_pred[:,i],mode="markers",name=f"run id {i}"),row=2,col=2)
fig.add_shape(type="line",x0=vcd_test.min(),y0=vcd_test.min(),x1=vcd_test.max(),y1=vcd_test.max(), layer='above', line=dict(dash='dash'),row=1,col=2)
fig.add_shape(type="line",x0=glc_test.min(),y0=glc_test.min(),x1=glc_test.max(),y1=glc_test.max(), layer='above', line=dict(dash='dash'),row=2,col=2)
fig.update_layout(title_text = "Observed vs Predicted",showlegend=False, height=1000)
fig.show()

## Discussion: Advantages and disadvantages of hybrid model


Pros:
*   Works well with small dataset
*   Describes the full dynamics
*   Extrapolation possible
*   Incorporate domain knowledge
*   ...


Cons:
*   Prone to failing due data mistakes
*   Units must be consistent
*   Prone to process dynamic misspecification
*   ...
